# Histogram of GOES-16, GOES-17, Empirical CDF, and AUC for Full disk samples in region of interest.

## Import libraries

In [1]:
import numpy as np
import scipy.stats as st
import itertools
import pandas as pd
# npPath = '/sharedData/scratch/all_npy3/'
# ncPath = '/sharedData/scratch/april_data/'
# acmPath = '/sharedData/scratch/all_npy3/'
# DATAPATH = '/sharedData/scratch/'
import re
import os
from os import path as op
import matplotlib.pyplot as plt
%matplotlib inline
import xarray as xr
import netCDF4
from pathlib import Path

import matplotlib.dates as mdates
from datetime import datetime

/scratch/adomakor412/conda/envs/MyEnv/lib/python3.6/site-packages/dask/config.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
import logging
logger = logging.getLogger()

## Define paths

In [5]:
storage = Path('../storage/')

## Functions for unfiltered cloud mask

In [7]:
def Rad2BT(rad, planck_fk1, planck_fk2, planck_bc1, planck_bc2):
    """Radiances to Brightness Temprature (using black body equation)"""
    invRad = np.array(rad)**(-1)
    arg = (invRad*planck_fk1) + 1.0
    T = (- planck_bc1+(planck_fk2 * (np.log(arg)**(-1))) )*(1/planck_bc2) 
    return T

In [9]:
def createUnfilteredPlotArray(ncFile,npFile,npPath):#Filtered Histrogram for cloud clear sky mask
    Tmean= []
    times = []
    for ncf, npf in zip(ncFile, npFile):
        imageBox = np.load(op.join(npPath,npf))
        myFile = xr.open_dataset(op.join(ncPath,ncf))
        planck_fk1 = float(myFile['planck_fk1'].data)
        planck_fk2 = float(myFile['planck_fk2'].data) 
        planck_bc1 = float(myFile['planck_bc1'].data)                       
        planck_bc2 = float(myFile['planck_bc2'].data)     
        T = Rad2BT(imageBox.mean(), planck_fk1, planck_fk2, planck_bc1, planck_bc2)
        tString = ncf[31:38]
        times.append(tString)
        Tmean.append(T)
    return times, Tmean

# Execution

In [24]:
template = '''\
    http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/\
    goes16_download.cgi?source=aws&satellite=\
    noaa-goes{SS}&domain=F&product=ABI-L1b-Rad&date=20{yy}-{mm}-{dd}&hour={hr}           
    '''

In [25]:
template

"    http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/    goes16_download.cgi?source=aws&satellite=    noaa-goes{SS}&domain=F&product=ABI-L1b-Rad&date=20{yy}-{mm}-{dd}&hour={hr}'            \n    "

In [44]:
template = 'http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/'   \
    + 'goes16_download.cgi?source=aws&satellite=' \
    + 'noaa-goes{SS}&domain=F&product=ABI-L1b-Rad&date=20{yy}-{mm}-{dd}&hour={hr}'

In [ ]:
for i in [8]:
    for j in range(11):
        band = str(i).zfill(2)
        date = 98 + j
        day = str(date).zfill(3)
        
        times17, mean17 = getTmean('17', npPath, band, day)
        times16, mean16 = getTmean('16', npPath, band, day)
        
        ptimes17 = [datetime.strptime(f"2019{t}", "%Y%j%H%M") for t in times17]
        ptimes16 = [datetime.strptime(f"2019{t}", "%Y%j%H%M") for t in times16]
        fig, ax1 = plt.subplots(figsize=(15,5))#constrained_layout=False
        _ = ax1.plot_date(ptimes17, mean17 , label="GOES-17", color='tab:red', marker='+', markersize = 16)
        _ = ax1.plot_date(ptimes16, mean16 , label='GOES-16', color='tab:blue', marker='+', markersize = 16, zorder=-1)
        _ = ax1.set_ylim(220,260)
        _ = ax1.tick_params('both', labelsize=18)
        _ = ax1.legend()
        _ = ax1.set_title(f'Mean Temperatures (K) for a 401 x 1001 Portion\n of Band 08 on 04-{str(8+j)}-2019', fontweight ='bold', fontsize = 22)
        _ = ax1.set_ylabel('Temperature (K)',fontsize = 16)
        _ = ax1.set_xlabel('time (hours)',fontsize = 16)
        _ = ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        
        
        _ = plt.savefig('..'+ op.join(DATAPATH,f"Mean_temp_unfiltered/04-{str(8+j).zfill(2)}-2019"))
